In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
!pwd


/workspaces/codespaces-blank/Chicken-Disease-Classification-Project


In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    sourcl_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.chicken_disease_classifier.constants import *  # noqa: F403
from src.chicken_disease_classifier.utils import read_yml,create_directories


In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH)->None:
        
        self.config=read_yml(config_filepath)
        self.params=read_yml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config= DataIngestionConfig(
            root_dir=config.root_dir,
            sourcl_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
import zipfile
from src.logger import logging
from src.chicken_disease_classifier.utils import get_size


In [13]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig)->None:
        self.config = config
        
        
    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                filename, headers=request.urlretrieve(
                url=self.config.sourcl_URL,
                filename=self.config.local_data_file
                )
                logging.info(f"{filename} downloaded with following info: \n{headers}")
        
            else:
                logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
        except Exception as e:
            logging.error(f"Error occurred in download_file of DataIngestion Class due to=>{e}")
      
     
    def unzip_file(self)->None:
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
        
        
                

In [15]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2025-01-11 15:38:55,511]: INFO : root : __init__ at 19 => Yaml file:config/config.yaml loaded Successfully
[2025-01-11 15:38:55,513]: INFO : root : __init__ at 19 => Yaml file:params.yaml loaded Successfully
[2025-01-11 15:38:55,514]: INFO : root : __init__ at 34 => Created directory at: artifacts
[2025-01-11 15:38:55,515]: INFO : root : __init__ at 34 => Created directory at: artifacts/data_ingestion
[2025-01-11 15:38:55,516]: INFO : root : 3277084385 at 16 => File already exists of size: ~11345 KB
